In [1]:
%%capture
import math
import statistics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
!pip install tensorflowjs
!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling

In [2]:
from google.colab import files
files.upload()
data = pd.read_csv("LCA_data.csv", sep=";")
dataset = data.values
input_data = dataset[:,0:6]
output_data = dataset[: ,6:16]

Saving LCA_data.csv to LCA_data.csv


In [3]:
def build_model():
  model = keras.Sequential([
    layers.Dense(16, activation="relu", input_shape=[6]),
    layers.Dense(5),
    layers.Dense(10)])
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mse", "accuracy"])
  return model
model = build_model()
history = model.fit(input_data, output_data, validation_split=0.2, epochs=1001, verbose=0, callbacks=[tfdocs.modeling.EpochDots()])
model.save("complete_model_saved.h5")
!mkdir output_folder
!tensorflowjs_converter --input_format keras complete_model_saved.h5 output_folder
!ls output_folder


Epoch: 0, accuracy:0.0000,  loss:267517552.0000,  mae:10676.3281,  mse:267517552.0000,  val_accuracy:0.0000,  val_loss:279094752.0000,  val_mae:10799.3242,  val_mse:279094752.0000,  
....................................................................................................
Epoch: 100, accuracy:1.0000,  loss:706910.8125,  mae:476.1949,  mse:706910.8125,  val_accuracy:1.0000,  val_loss:1276730.0000,  val_mae:638.4736,  val_mse:1276730.0000,  
....................................................................................................
Epoch: 200, accuracy:1.0000,  loss:421089.5000,  mae:361.4267,  mse:421089.5000,  val_accuracy:1.0000,  val_loss:773185.7500,  val_mae:490.1812,  val_mse:773185.7500,  
....................................................................................................
Epoch: 300, accuracy:1.0000,  loss:170794.2969,  mae:229.3506,  mse:170794.2969,  val_accuracy:1.0000,  val_loss:473535.3125,  val_mae:375.6222,  val_mse:473535.3125,  
....

In [4]:
Asphalt = float(input("optimal asphalt binder content determined for the WMA without RCA, %: "))
Additive = float(input("chemical additive content by asphalt binder weight, %: "))
TR = float(input("temperature reduction achieved by the chemical additive implemented, °C: "))
HD = float(input("hauling distance of the RCA, km: "))
WC = float(input("average water content in aggregates (natural and RCA), %: "))

optimal asphalt binder content determined for the WMA without RCA, %: 4.4
chemical additive content by asphalt binder weight, %: 0.3
temperature reduction achieved by the chemical additive implemented, °C: 40
hauling distance of the RCA, km: 0
average water content in aggregates (natural and RCA), %: 3


In [5]:
matrix_size = np.zeros((46, 10))
matrix = np.zeros_like(matrix_size, dtype=float)
RCA = 0
while RCA <= 45:
  matrix[RCA, :] = model.predict(np.array([[RCA, Asphalt, Additive, TR, HD, WC]]))
  RCA = RCA + 1
vector_results = np.arange(0, 10, 1)
z = 2.58 # critical z-value with confidence level of 99%

In [6]:
for i in range (0, 10):
  min, optimal = np.amin(matrix[:, i]), 0
  while (optimal<45):
    if (matrix[:, i][optimal] == min):
      break
    else:
      optimal = optimal + 1
  dev_stan = statistics.stdev(matrix[:, i])
  upper_limit = matrix[:, i][optimal] + (z*dev_stan/math.sqrt(46))
  while (optimal<45):
    lower_limit = matrix[:, i][optimal+1] - (z*dev_stan/math.sqrt(46))
    if (upper_limit > lower_limit):
      optimal = optimal + 1
    else:
      break
  vector_results[i] = optimal

In [7]:
optimal_RCA = np.amin(vector_results)
result_with_optimal_RCA = model.predict(np.array([[optimal_RCA, Asphalt, Additive, TR, HD, WC]]))
result_with_optimal_RCA = result_with_optimal_RCA[0]
print("From an environmental approach, the maximum coarse aggregate content that can be replaced by RCA is: " + str(optimal_RCA) + "%")
print("The production of 1 ton of WMA (with the optimal coarse RCA content) generates: ")
print("Ozone depletion: " + str(int(result_with_optimal_RCA[0])) + " mg CFC-11eq")
print("Global warming: " + str(int(result_with_optimal_RCA[1])) + " kg CO2eq")
print("Smog: " + str(int(result_with_optimal_RCA[2])) + " kg O3eq")
print("Acidification: " + str(int(result_with_optimal_RCA[3])) + " kg SO2eq")
print("Eutrophication: " + str(int(result_with_optimal_RCA[4])) + " g Neq")
print("Carcinogenic: " + str(int(result_with_optimal_RCA[5])) + " E-6 CTUh")
print("Non-carcinogenic: " + str(int(result_with_optimal_RCA[6])) + " E-6 CTUh")
print("Respiratory effects: " + str(int(result_with_optimal_RCA[7])) + " g PM2.5eq")
print("Ecotoxicity: " + str(int(result_with_optimal_RCA[8])) + " CTUe")
print("Fossil fuel depletion: " + str(int(result_with_optimal_RCA[9])) + " MJ surplus")

From an environmental approach, the maximum coarse aggregate content that can be replaced by RCA is: 10%
The production of 1 ton of WMA (with the optimal coarse RCA content) generates: 
Ozone depletion: 3862 mg CFC-11eq
Global warming: 18342 kg CO2eq
Smog: 754 kg O3eq
Acidification: 120 kg SO2eq
Eutrophication: 9949 g Neq
Carcinogenic: 136 E-6 CTUh
Non-carcinogenic: 962 E-6 CTUh
Respiratory effects: 8826 g PM2.5eq
Ecotoxicity: 28777 CTUe
Fossil fuel depletion: 34733 MJ surplus
